# Setup

In [1]:
import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
chapter_dir = r"./" if chapter in os.listdir() else os.getcwd().split(chapter)[0]
sys.path.append(chapter_dir + f"{chapter}/exercises")

import os
os.environ["ACCELERATE_DISABLE_RICH"] = "1"
import sys
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor
from torch.utils.data import DataLoader
import numpy as np
import einops
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
import itertools
import random
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
from typing import List, Union, Optional, Tuple, Callable, Dict
import typeguard
from functools import partial
# from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import f1_score as multiclass_f1_score
import copy
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from tqdm.notebook import tqdm
from dataclasses import dataclass
from rich import print as rprint
import pandas as pd

# Make sure exercises are in the path
# exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
# section_dir = exercises_dir / "part6_othellogpt"
# section_dir = "interpretability"
# if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow
from neel_plotly import scatter, line
from generate_patches import generate_patch
from pprint import pprint
from utils import plot_game
from training_utils import get_state_stack_num_flipped
from utils import plot_probe_outputs
from utils import seq_to_state_stack
# import part6_othellogpt.tests as tests

t.manual_seed(42)

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

cfg = HookedTransformerConfig(
    n_layers = 8,
    d_model = 512,
    d_head = 64,
    n_heads = 8,
    d_mlp = 2048,
    d_vocab = 61,
    n_ctx = 59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)
model = HookedTransformer(cfg)

sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "synthetic_model.pth")
# champion_ship_sd = utils.download_file_from_hf("NeelNanda/Othello-GPT-Transformer-Lens", "championship_model.pth")
model.load_state_dict(sd)

# An example input
sample_input = t.tensor([[
    20, 19, 18, 10,  2,  1, 27,  3, 41, 42, 34, 12,  4, 40, 11, 29, 43, 13, 48, 56,
    33, 39, 22, 44, 24,  5, 46,  6, 32, 36, 51, 58, 52, 60, 21, 53, 26, 31, 37,  9,
    25, 38, 23, 50, 45, 17, 47, 28, 35, 30, 54, 16, 59, 49, 57, 14, 15, 55, 7
]]).to(device)

# The argmax of the output (ie the most likely next move from each position)
sample_output = t.tensor([[
    21, 41, 40, 34, 40, 41,  3, 11, 21, 43, 40, 21, 28, 50, 33, 50, 33,  5, 33,  5,
    52, 46, 14, 46, 14, 47, 38, 57, 36, 50, 38, 15, 28, 26, 28, 59, 50, 28, 14, 28,
    28, 28, 28, 45, 28, 35, 15, 14, 30, 59, 49, 59, 15, 15, 14, 15,  8,  7,  8
]]).to(device)

assert (model(sample_input).argmax(dim=-1) == sample_output.to(device)).all()

# os.chdir(section_dir)
section_dir = Path.cwd()
sys.path.append(str(section_dir))
print(section_dir.name)

OTHELLO_ROOT = (section_dir / "othello_world").resolve()
OTHELLO_MECHINT_ROOT = (OTHELLO_ROOT / "mechanistic_interpretability").resolve()

# if not OTHELLO_ROOT.exists():
#     !git clone https://github.com/likenneth/othello_world

sys.path.append(str(OTHELLO_MECHINT_ROOT))

from mech_interp_othello_utils import (
    plot_board,
    plot_single_board,
    plot_board_log_probs,
    to_string,
    to_int,
    int_to_label,
    string_to_label,
    OthelloBoardState
)

# Load board data as ints (i.e. 0 to 60)
board_seqs_int = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_int_small.npy"), dtype=t.long)
# Load board data as "strings" (i.e. 0 to 63 with middle squares skipped out)
board_seqs_string = t.tensor(np.load(OTHELLO_MECHINT_ROOT / "board_seqs_string_small.npy"), dtype=t.long)

assert all([middle_sq not in board_seqs_string for middle_sq in [27, 28, 35, 36]])
assert board_seqs_int.max() == 60

num_games, length_of_game = board_seqs_int.shape

# Define possible indices (excluding the four center squares)
stoi_indices = [i for i in range(64) if i not in [27, 28, 35, 36]]

# Define our rows, and the function that converts an index into a (row, column) label, e.g. `E2`
alpha = "ABCDEFGH"

def to_board_label(i):
    return f"{alpha[i//8]}{i%8}"

# Get our list of board labels
board_labels = list(map(to_board_label, stoi_indices))
full_board_labels = list(map(to_board_label, range(64)))

def plot_square_as_board(state, diverging_scale=True, **kwargs):
    """Takes a square input (8 by 8) and plot it as a board. Can do a stack of boards via facet_col=0"""
    kwargs = {
        "y": [i for i in alpha],
        "x": [str(i) for i in range(8)],
        "color_continuous_scale": "RdBu" if diverging_scale else "Blues",
        "color_continuous_midpoint": 0. if diverging_scale else None,
        "aspect": "equal",
        **kwargs
    }
    imshow(state, **kwargs)

start = 30000
num_games = 50
focus_games_int = board_seqs_int[start : start + num_games]
focus_games_string = board_seqs_string[start: start + num_games]

focus_logits, focus_cache = model.run_with_cache(focus_games_int[:, :-1].to(device))
focus_logits.shape

def one_hot(list_of_ints, num_classes=64):
    out = t.zeros((num_classes,), dtype=t.float32)
    out[list_of_ints] = 1.
    return out

focus_states = np.zeros((num_games, 60, 8, 8), dtype=np.float32)
focus_valid_moves = t.zeros((num_games, 60, 64), dtype=t.float32)

for i in (range(num_games)):
    board = OthelloBoardState()
    for j in range(60):
        board.umpire(focus_games_string[i, j].item())
        focus_states[i, j] = board.state
        focus_valid_moves[i, j] = one_hot(board.get_valid_moves())

print("focus states:", focus_states.shape)
print("focus_valid_moves", tuple(focus_valid_moves.shape))

# full_linear_probe = t.load(OTHELLO_MECHINT_ROOT / "main_linear_probe.pth", map_location=device)

linear_probe2 = t.load("probes/linear/resid_6_linear.pth")

rows = 8
cols = 8
options = 3
assert linear_probe2.shape == (1, cfg.d_model, rows, cols, options)

black_to_play_index = 0
white_to_play_index = 1
blank_index = 0
their_index = 1
my_index = 2

# Creating values for linear probe (converting the "black/white to play" notation into "me/them to play")

'''LAYER = 6
game_index = 0
move = 29'''

BLANK1 = 0
BLACK = 1
WHITE = -1

# MINE = 0
# YOURS = 1
# BLANK2 = 2

EMPTY = 0
YOURS = 1
MINE = 2

/hpi/fs00/home/jim.maar/miniconda3/envs/othello-env/lib/python3.11/site-packages/accelerate/utils/imports.py:274: UserWarning: `ACCELERATE_DISABLE_RICH` is deprecated and will be removed in v0.22.0 and deactivated by default. Please use `ACCELERATE_ENABLE_RICH` if you wish to use `rich`.
  warnings.warn(


interpretability
interpretability
focus states: (50, 60, 8, 8)
focus_valid_moves (50, 60, 64)


## More Imports

In [2]:
from utils import visualize_game

# Code

In [3]:
# Load Probes
linear_probes = []
flipped_probes = []
for layer in range(8):
    linear_probe = t.load(f"probes/linear/resid_{layer}_linear.pth").to(device)
    flipped_probe = t.load(f"probes/flipped/resid_{layer}_flipped.pth").to(device)
    linear_probes.append(linear_probe)
    flipped_probes.append(flipped_probe)
print(len(linear_probes), len(flipped_probes))

8 8


In [4]:
def get_feature_logits(resid : Float[Tensor, "batch pos d_model"], layer : Int, softmax : bool = False) -> Tuple[Float[Tensor, "model batch seq rows cols options"], Float[Tensor, "model batch seq rows cols options"]]:
    """
    resid: [batch pos d_model]
    layer: Int
    softmax: bool
    output:
        Tuple[Float[Tensor, "model batch seq rows cols options"], Float[Tensor, "model batch seq rows cols options"]]
        The first tensor is the logits for the board, the second tensor is the logits for the flipped board
    """
    linear_probe = linear_probes[layer]
    flipped_probe = flipped_probes[layer]
    board_logits = einops.einsum(resid, linear_probe, 'batch seq d_model, modes d_model rows cols options -> modes batch seq rows cols options')[0]
    flipped_logits = einops.einsum(resid, flipped_probe, 'batch seq d_model, modes d_model rows cols options -> modes batch seq rows cols options')[0]
    if softmax:
        board_logits = F.softmax(board_logits, dim=-1)
        flipped_logits = F.softmax(flipped_logits, dim=-1)
    return board_logits, flipped_logits

In [5]:
    
# Plan: For Layer 1 to 7. Assume Mine Yours comes from Last Flipped, What accuracy do I get?
# result = t.zeros((7, 2), dtype=t.float32) # Layer, (mine vs yours)
# TODO: Mine and Yours seperately
# TODO: position seperately
def test_last_flipped_accuracy(end_pos : Int):
    flipped_thresh = 0.9
    how_far_back = 15
    for layer in range(1, 7):
        resid_pre = focus_cache["resid_pre", layer]
        resid_post = focus_cache["resid_mid", layer]
        # shape: [batch pos rows cols options]
        mine_yours_logits_pre, flipped_logits_pre = get_feature_logits(resid_pre, layer-1)
        mine_yours_logits_post, flipped_logits_post = get_feature_logits(resid_post, layer)
        batch_size, seq_len, _, _, _ = mine_yours_logits_post.shape
        last_flipped_values = t.zeros_like(mine_yours_logits_post)
        last_flipped_values[:, :, :, :, 0] = 1
        # TODO: Not sure if this is correct
        for pos1 in range(seq_len):
            # MINE
            for pos in range(max([pos1 + 1 - how_far_back, 0]), pos1 + 1):
                # flipped_diff = flipped_logits_pre[:, pos, :, :, 0] - flipped_logits_pre[:, pos, :, :, 1]
                flipped_diff = flipped_logits_pre.softmax(dim=-1)[:, pos, :, :, 0]
                was_flipped = flipped_diff > flipped_thresh
                # if pos1 % 2 != pos % 2:
                # last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre[:, pos][was_flipped]
                # print(layer, pos1, pos)
                '''if layer == 1 and pos1 == 5 and pos == 4:
                    print(f"Debug: {flipped_diff.shape}, {was_flipped.shape}, {last_flipped_values.shape}")
                    print(flipped_diff[1, :, :])
                    print(was_flipped[1, :, :])
                    print(last_flipped_values[1, pos1, :, :, 1])'''
                if pos1 % 2 == pos % 2:
                    last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre[:, pos][was_flipped]
                else:
                    mine_yours_logits_pre_copy = mine_yours_logits_pre.clone()
                    # print(mine_yours_logits_pre.shape, mine_yours_logits_pre_copy.shape, was_flipped.shape)
                    mine_yours_logits_pre_copy[:, :, :, 1], mine_yours_logits_pre_copy[:, :, :, 2] = mine_yours_logits_pre[:, :, :, 2], mine_yours_logits_pre[:, :, :, 1]
                    last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre_copy[:, pos][was_flipped]
        # Calculate Accuracy
        # TODO: Dises Mess richtig machen,
        # TODO: Prediction für flipped von diesem Move auch rein machen (Einfach for loop erhöhen. und aus for loop die 2 raus nehmen)
        # TODO: Danach das Ding in viele nützliche Funktionen aufteilen
        last_flipped_values_pred = last_flipped_values.argmax(dim=-1)
        real_values = mine_yours_logits_post.argmax(dim=-1)
        # real_values shape : [batch pos, rows, cols]()
        # get number of zero_preds
        num_preds_per_pos = (t.ones((batch_size, seq_len)) * 64).to(device) - einops.reduce((last_flipped_values_pred == 0), 'batch pos rows cols -> batch pos', reduction='sum')
        num_positions_with_preds = (num_preds_per_pos > 0).sum()
        num_preds_per_pos[num_preds_per_pos == 0] = 1
        last_flipped_values_pred[last_flipped_values_pred == 0] = -1
        acc_correct_per_pos = einops.reduce((last_flipped_values_pred == real_values).float(), 'batch pos rows cols -> batch pos', reduction='sum') / num_preds_per_pos
        acc = acc_correct_per_pos.mean()
        print(f"Layer: {layer}, Accuracy: {acc.item()}")

test_last_flipped_accuracy(20)

Layer: 1, Accuracy: 0.5641120672225952


Layer: 2, Accuracy: 0.5826246738433838
Layer: 3, Accuracy: 0.5996385812759399
Layer: 4, Accuracy: 0.6073006987571716
Layer: 5, Accuracy: 0.6164507865905762
Layer: 6, Accuracy: 0.6075239181518555


In [14]:
    
# TODO: Print out the Mistakes, so I can visualize them
# TODO: Print out the Tiles where the mistake is
# TODO: Import shit to visualize
# TODO: Visualize
# TODO: Different Layers: Different Threshholds, mabe different positions ???
# TODO: Add the Flipping Circuit

@dataclass
class Parameters():
    flipped_thresh : Float = 0.2
    how_far_back : Int = 15
    start_pos : Int = 10
    end_pos : Int = 20
    layers : Int = 7
    evaluation_module : str = "attn_out"
    include_current_pos : bool = False

def get_predictions(params : Parameters, resid_pre: Float[Tensor, "batch pos d_model"], layer : Int) -> Float[Tensor, "batch pos rows cols options"]:
    # batch pos rows cols option
    mine_yours_logits_pre, flipped_logits_pre = get_feature_logits(resid_pre, layer-1)
    _, seq_len, _, _, _ = mine_yours_logits_pre.shape
    last_flipped_values = t.zeros_like(mine_yours_logits_pre)
    last_flipped_values[:, :, :, :, 0] = 1
    for pos1 in range(params.start_pos, params.end_pos):
        if params.include_current_pos:
            local_end_pos = pos1 + 1
        else:
            local_end_pos = pos1
        for pos in range(max([local_end_pos - params.how_far_back, 0]), local_end_pos):
            flipped_diff = flipped_logits_pre.softmax(dim=-1)[:, pos, :, :, 0]
            was_flipped = flipped_diff > params.flipped_thresh
            if pos1 % 2 == pos % 2:
                last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre[:, pos][was_flipped]
            else:
                mine_yours_logits_pre_copy = mine_yours_logits_pre.clone()
                # print(mine_yours_logits_pre_copy[0, ])
                mine_yours_logits_pre_copy[:, pos, :, :, 1], mine_yours_logits_pre_copy[:, pos, :, :, 2] = mine_yours_logits_pre[:, pos, :, :, 2], mine_yours_logits_pre[:, pos, :, :, 1]
                last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre_copy[:, pos][was_flipped]
            # last_flipped_values[:, pos1][was_flipped] = mine_yours_logits_pre[:, pos][was_flipped]
    return last_flipped_values

def print_first_mistakes(pred : Float[Tensor, "batch pos rows cols"], real : Float[Tensor, "batch pos rows cols"], num_mistakes : Int):
    batch_size, seq_len, rows, cols = pred.shape
    mistakes = ((pred != real) & (pred != 0))
    for i in range(batch_size):
        for j in range(seq_len):
            for row in range(rows):
                for col in range(cols):
                    if not mistakes[i, j, row, col]:
                        continue
                    tile_label = string_to_label(row*8 + col)
                    print(f"Game: {i}, Pos: {j}, Tile: {tile_label}")
                    num_mistakes -= 1
                    if num_mistakes == 0:
                        return

def get_accuracy(last_flipped_values : Float[Tensor, "batch pos rows cols options"], resid_post : Float[Tensor, "batch pos d_model"], layer : Int) -> Float[Tensor, ""]:
    batch_size, seq_len, _, _, _ = last_flipped_values.shape
    mine_yours_logits_post, _ = get_feature_logits(resid_post, layer)
    last_flipped_values_pred = last_flipped_values.argmax(dim=-1)
    real_values = mine_yours_logits_post.argmax(dim=-1)
    print_first_mistakes(last_flipped_values_pred, real_values, 5)
    # real_values shape : [batch pos, rows, cols]()
    num_preds_per_pos = (t.ones((batch_size, seq_len)) * 64).to(device) - einops.reduce((last_flipped_values_pred == 0), 'batch pos rows cols -> batch pos', reduction='sum')
    num_positions_with_preds = (num_preds_per_pos > 0).sum(dim=-1)
    num_preds_per_pos[num_preds_per_pos == 0] = 1
    last_flipped_values_pred[last_flipped_values_pred == 0] = -1
    acc_correct_per_pos = einops.reduce((last_flipped_values_pred == real_values).float(), 'batch pos rows cols -> batch pos', reduction='sum') / num_preds_per_pos
    acc = (acc_correct_per_pos.sum(dim=-1)/num_positions_with_preds).mean()
    return acc


def test_last_flipped_accuracy(params : Parameters):
    for layer in range(1, params.layers):
        resid_pre = focus_cache["resid_pre", layer]
        resid_post = focus_cache[params.evaluation_module, layer]
        # shape: [batch pos rows cols options]
        last_flipped_values = get_predictions(params, resid_pre, layer)
        acc = get_accuracy(last_flipped_values, resid_post, layer)
        print(f"Layer: {layer}, Accuracy: {acc.item()}")

params = Parameters()
params.layers = 2
params.start_pos = 20
params.end_pos = 30

test_last_flipped_accuracy(params)

Game: 0, Pos: 20, Tile: E4
Game: 0, Pos: 21, Tile: E3
Game: 0, Pos: 21, Tile: E4
Game: 0, Pos: 22, Tile: D3
Game: 0, Pos: 22, Tile: D4
Layer: 1, Accuracy: 0.8634616136550903


In [9]:
from utils import VisualzeBoardArguments
from utils import visualize_game

In [19]:
vis_args = VisualzeBoardArguments()
vis_args.start_pos = 4
vis_args.end_pos = 23
vis_args.layers = 6
vis_args.include_attn_only = True
vis_args.include_mlp_only = False
# vis_args.static_image = True

clean_input_str = focus_games_string[0, :59]

visualize_game(clean_input_str, vis_args, model)

torch.Size([8, 2, 59, 8, 8])
